In [7]:
import os
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np

# Define the augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=40, p=0.7),
    A.RandomCrop(height=200, width=200, p=0.5),  # Ensures crop of exactly 200x200
    A.Resize(height=224, width=224),  # Resize it back to 224x224
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.0, 0.0, 0.0), std=(1.0, 1.0, 1.0)),
    ToTensorV2()
])

# Function to apply augmentation and save image
def augment_and_save(image_path, save_img_dir, augment_count=5):
    # Load the image
    img = cv2.imread(image_path)

    img_name = os.path.basename(image_path).split('.')[0]

    for i in range(augment_count):
        # Apply augmentation
        augmented = transform(image=img)
        augmented_img = augmented['image'].cpu().numpy().transpose(1, 2, 0)

        # Convert back to original scale (unnormalize) for the image
        augmented_img = (augmented_img * 255).astype(np.uint8)

        # Save the augmented image with updated filenames
        new_img_name = f"{img_name}_aug_{i}.jpg"
        cv2.imwrite(os.path.join(save_img_dir, new_img_name), augmented_img)

# Directory for images
img_dir = 'images'

# Directory for saving augmented images
save_img_dir = 'img_aug'

# Create output directory if it doesn't exist
os.makedirs(save_img_dir, exist_ok=True)

# Apply augmentation and save for each image
for img_name in os.listdir(img_dir):
    img_path = os.path.join(img_dir, img_name)

    # Perform augmentation only on images
    augment_and_save(img_path, save_img_dir)
